In [1]:
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import *
from torchvision.utils import save_image
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
from project.data_utils import *
from project.models.ambiguous_generator import *
from project.models.cvae import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
print(device)

cuda


/home/nislah/.vmgr_repo/dev-2021-02-py38/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


# EMNIST example

In [2]:
with open('dataset/emnist_params.yaml','r') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
n_classes = 26
latent_dim = 4
enc_layers = [28*28, 512]
dec_layers = [512, 28*28]
ckpt_path = params['ckpt_path']
img_path = params['img_path']
model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
ckpt = torch.load(ckpt_path)
model.load_state_dict(ckpt['state_dict'])
model.eval()
encoder,decoder = model.encoder,model.decoder

In [3]:
dataset = datasets.EMNIST(root='/share/datasets', download=False, train=True, split='letters', transform=transforms.Compose([transforms.ToTensor()]))
generator = EMNISTGenerator(encoder, decoder, DataLoader(dataset, batch_size=2, shuffle=True), n_classes=n_classes, device=device)
letters='abcdefghijklmnopqrstuvwxyz'
idx = lambda x: letters.index(x)
pure_pairs = np.array([(idx('c'), idx('o')), (idx('c'), idx('e'))])
dataset = AmbiguousDataset(generator, pure_pairs, blend=0.5, n_classes=n_classes)
ambiguousDataLoader = DataLoader(dataset, batch_size=64, shuffle=True)

x,t = next(iter(ambiguousDataLoader))
save_image(x,img_path,nrow=8)

# MNIST example

In [4]:
with open('dataset/save_dict.yaml','r') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
n_classes = params['n_classes']
img_path=params['img']
pure_pairs=np.array([(3,8),(8,3),(3,5),(5,3),(5,8),(8,5),(0,6),(6,0)])
encoder = torch.load(params['enc']).to(device)
decoder = torch.load(params['dec']).to(device)
encoder.eval(),decoder.eval()

(Encoder(
   (l1): Linear(in_features=794, out_features=500, bias=True)
   (l2): Linear(in_features=500, out_features=4, bias=True)
 ),
 Decoder(
   (l1): Linear(in_features=12, out_features=500, bias=True)
   (l2): Linear(in_features=500, out_features=784, bias=True)
 ))

In [6]:
dataset = datasets.MNIST(root='/share/datasets', download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))
generator = MNISTGenerator(encoder, decoder, DataLoader(dataset, batch_size=2, shuffle=True), n_classes=n_classes, device=device)
dataset = AmbiguousDataset(generator, pure_pairs, transform=transforms.Compose([transforms.Resize((32,32))]), n_classes=n_classes, blend=0.5)
ambiguousDataLoader = DataLoader(dataset, batch_size=64, shuffle=True)

x,t = next(iter(ambiguousDataLoader))
save_image(x,img_path,nrow=8)